In [1]:
import requests
import json
import pandas as pd

In [2]:
%%capture
requests.post('https://maker.ifttt.com/trigger/heroku_app/with/key/{KEY}', params={'value1':'DJIA Rankings'})

In [3]:
def get_json(ticker):
    output = {}
    output['ticker'] = ticker

    quote = requests.get('https://finnhub.io/api/v1/quote?symbol='+ticker+'&token={KEY}')
    quote_json = json.loads(str(quote.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    output['quote'] = quote_json
        
    aggr = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol='+ticker+'&resolution=D&token={KEY}')
    aggr_json = json.loads(str(aggr.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    output['technicalAnalysis'] = aggr_json

    senti = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol='+ticker+'&token={KEY}')
    senti_json = json.loads(str(senti.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    output['sentiment'] = senti_json
        
    fins = requests.get('https://finnhub.io/api/v1/stock/metric?symbol='+ticker+'&metric=all&token={KEY}')
    fins_json = json.loads(str(fins.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    fins_json = fins_json["metric"]
    output['financials'] = fins_json

    return json.dumps(output, indent=2)

In [4]:
def get_quote(ticker):
    quote = get_json(ticker)
    quote_json = eval(quote)
    
    close = quote_json['quote']['c']
    prevClose = quote_json.get('quote').get('pc')
    change = round((close/prevClose-1)*100, 2)
    signal = quote_json.get('technicalAnalysis').get('technicalAnalysis').get('signal')
    articlesLastWk = quote_json.get('sentiment').get('buzz').get('articlesInLastWeek')
    weeklyAvg = int(quote_json.get('sentiment').get('buzz').get('weeklyAverage')+0.5)
    newsScore = quote_json.get('sentiment').get('companyNewsScore')
    sectorAvgNewsScore = quote_json.get('sentiment').get('sectorAverageNewsScore')
    buzz = quote_json.get('sentiment').get('buzz').get('buzz')
    bullishPercent = round(quote_json.get('sentiment').get('sentiment').get('bullishPercent')*100, 2)
    sectorAvgBullishPercent = round(quote_json.get('sentiment').get('sectorAverageBullishPercent')*100, 2)
    
    return {'ticker':ticker, 'change':change,'close':close,'prevClose':prevClose,'signal':signal,'articlesLastWk':articlesLastWk,'weeklyAvg':weeklyAvg,'newsScore':newsScore,'sectorAvgNewsScore':sectorAvgNewsScore,'buzz':buzz,'bullishPercent':bullishPercent,'sectorAvgBullishPercent':sectorAvgBullishPercent}

In [5]:
df = pd.DataFrame(columns=['ticker','change','close','prevClose','signal','articlesLastWk','weeklyAvg','newsScore','sectorAvgNewsScore','buzz','bullishPercent','sectorAvgBullishPercent'])

for ticker in ['JPM', 'IBM', 'UNH', 'PG', 'BA', 'VZ', 'GS', 'INTC', 'HD', 'MCD', 'CSCO', 'V', 'MSFT', 'KO', 'AXP', 'AMGN', 'MRK', 'CVX', 'MMM', 'TRV', 'JNJ', 'DIS', 'HON', 'CAT', 'NKE', 'CRM', 'WBA', 'AAPL', 'DOW', 'WMT']:
    df = df.append(get_quote(ticker), ignore_index = True)
    
df = df.sort_values(by=['bullishPercent','newsScore','articlesLastWk','change'], ascending=False)
df = df.reset_index(drop=True)
df.head(len(df))

,ticker,change,close,prevClose,signal,articlesLastWk,weeklyAvg,newsScore,sectorAvgNewsScore,buzz,bullishPercent,sectorAvgBullishPercent
0,NKE,-0.30,127.99,128.37,neutral,22,23,0.9166,0.5222,0.9670,100.00,62.53
1,HD,0.29,276.84,276.04,neutral,14,21,0.9166,0.5222,0.6829,100.00,62.53
2,MRK,-1.08,77.99,78.84,neutral,35,26,0.8608,0.5022,0.9809,100.00,58.25
3,AMGN,-0.64,223.46,224.89,sell,20,15,0.8608,0.5022,0.9954,100.00,58.25
4,HON,-2.01,166.75,170.17,neutral,15,17,0.6178,0.5050,0.8823,100.00,60.18
5,UNH,-0.79,320.51,323.06,neutral,15,25,0.5000,0.5022,0.6122,100.00,58.25
6,CSCO,-2.12,36.87,37.67,sell,10,18,0.5000,0.5181,0.5479,100.00,64.56
7,MSFT,1.51,213.25,210.08,buy,184,176,1.0000,0.5181,1.0469,96.29,64.56
8,JNJ,-0.57,143.15,143.97,sell,60,91,0.6346,0.5022,0.6611,83.33,58.25
9,KO,0.20,49.90,49.80,neutral,79,42,0.6666,0.5222,1.8809,75.00,62.53
